# Training a brain age model

## Packages importation

In [1]:
import nibabel as nib
import pandas as pd
import glob 
import math
import numpy as np
import tensorflow as tf
import datetime
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# Own bibli
from callbacks import SaveMetrics
from generators import generator_mri_regression_norm, generator_mri_regression_norm_aug
from regression import model_Cole

# Tensorflow special importation
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint

ImportError: cannot import name 'coordinateTransformWrapper' from 'data_augmentation' (/NAS/deathrow/morgan/Documents/dev/brain_age/data_augmentation/GitHub/data_augmentation.py)

## Hyperparameters

In [ ]:
SIZE = (182, 218, 182, 1)
LR = 0.001
BATCH_SIZE = 8

## Data importation

In [ ]:
data_frame_train = pd.read_csv('YOUR_TRAIN_SET')
data_frame_test = pd.read_csv('YOUR_TEST_SET')

## Callback initialization

In [ ]:
# Call back initialization
training_path = 'YOUR_TRAINING_PATH'
os.system('mkdir '+training_path)

save_metrics = SaveMetrics(training_path)

filepath=training_path+"/model_saved/"
os.system("mkdir "+filepath)

checkpoint = ModelCheckpoint(filepath+"weights-{epoch:02d}-{val_mae:.2f}.hdf5", monitor='val_mae', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, save_metrics]

## Generator initialization

In [ ]:
model = model_Cole(SIZE)

my_generator_train = generator_mri_regression_norm_aug(list_path=data_frame_train.loc[:, ['path', 'age']], batch_size=BATCH_SIZE, shuffle=True, norm=True)

my_generator_test = generator_mri_regression_norm(list_path=data_frame_test.loc[:, ['path', 'age']], batch_size=BATCH_SIZE, shuffle=True, norm=True)

model.compile(loss=['mae'],
              optimizer=Adagrad(LR),
              metrics=['mae'])

## Training

In [ ]:
model.fit_generator(generator=my_generator_train.loader(),
                    steps_per_epoch=my_generator_train.get_len(),
                    epochs=300, 
                    verbose=1,
                    validation_data=my_generator_test.loader(),
                    validation_steps=my_generator_test.get_len(), 
                    validation_freq=1,
                    shuffle=True,
                    initial_epoch=0,
                    callbacks=callbacks_list)